In [1]:
import tensorflow as tf
import numpy as np
import gym
import datetime
from tqdm import tqdm
import random
from go_ai import data, metrics, mcts, models, policies
import matplotlib.pyplot as plt
import os
import multiprocessing as mp

# Hyperparameters

In [2]:
BOARD_SIZE = 7

In [3]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 64
NUM_EVAL_GAMES = 32

In [4]:
EPISODES_DIR = './data/'

In [5]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-3

In [6]:
WEIGHTS_DIR = 'model_weights/'
CHECKPOINT_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
TMP_WEIGHTS_PATH = WEIGHTS_DIR + 'tmp.h5'
LOAD_SAVED_MODELS = True

In [7]:
NUM_WORKERS = mp.cpu_count()

# Go Environment
Train on a small board with heuristic reward for fast training and efficient debugging

In [8]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Metrics and Tensorboard

In [9]:
!rm -rf ./logs/

Metrics

In [10]:
tb_metrics = {}
for metric_key in ['val_loss', 'move_loss']:
    tb_metrics[metric_key] = tf.keras.metrics.Mean('{}'.format(metric_key), 
                                                   dtype=tf.float32)
tb_metrics['pred_win_acc'] = tf.keras.metrics.Accuracy()

Tensorboard

In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/actor_critic/{}/main'.format(current_time)
summary_writer = tf.summary.create_file_writer(log_dir)    

# Machine Learning Models

In [12]:
actor_critic = models.make_actor_critic(BOARD_SIZE)

In [13]:
_ = tf.keras.utils.plot_model(actor_critic, to_file='logs/model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [14]:
actor_critic.summary()

Model: "actor_critic"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 7, 7, 6)]    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 7, 7, 64)     3520        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 7, 7, 64)     256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 7, 7, 64)     0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [15]:
if LOAD_SAVED_MODELS:
    actor_critic.load_weights(CHECKPOINT_PATH)
    print("Loaded saved models")
else:
    actor_critic.save_weights(CHECKPOINT_PATH)
    print("Synced checkpoint with temp")
    
actor_critic.save_weights(TMP_WEIGHTS_PATH)

Loaded saved models


# Policies

In [16]:
temp_policy_args = {
    'mode': 'actor_critic',
    'model_path': TMP_WEIGHTS_PATH,
}

In [17]:
checkpoint_policy_args = {
    'mode': 'actor_critic',
    'model_path': CHECKPOINT_PATH,
}

In [18]:
random_policy_args = {
    'mode': 'random'
}

In [19]:
greedy_policy_args = {
    'mode': 'greedy'
}

# Value Function

In [20]:
tmp_forward_func_args = {
    'weights_path': TMP_WEIGHTS_PATH
}

# Demo Trajectories

Symmetries

In [21]:
%%time
metrics.plot_symmetries(go_env, actor_critic, 'logs/symmetries.jpg')

CPU times: user 1.49 s, sys: 120 ms, total: 1.61 s
Wall time: 1.02 s


Plot a whole game trajectory

In [22]:
%%time
data.make_episodes(BOARD_SIZE, temp_policy_args, temp_policy_args, tmp_forward_func_args, 1, num_workers=1, 
                   outdir=EPISODES_DIR)

Episodes: 100%|██████████| 1/1 [00:00<00:00, 1174.88it/s, 0.0%]

CPU times: user 5.45 s, sys: 237 ms, total: 5.69 s
Wall time: 5.29 s


0.0

In [23]:
%%time
fig = metrics.gen_traj_fig(go_env, actor_critic)
fig.savefig('logs/a_trajectory.jpg')
plt.close()

CPU times: user 14.5 s, sys: 597 ms, total: 15.1 s
Wall time: 14.1 s


# Train

In [24]:
actor_critic_opt = tf.keras.optimizers.Adam(LEARNING_RATE)

In [25]:
for iteration in range(ITERATIONS):
    for optimize_mode in ['critic', 'actor']:
        # Make and write out the episode data
        data.make_episodes(BOARD_SIZE, temp_policy_args, temp_policy_args, tmp_forward_func_args,
                           EPISODES_PER_ITERATION, num_workers=NUM_WORKERS, outdir=EPISODES_DIR)
        # Read in the episode data
        np_data = data.episodes_from_dir(EPISODES_DIR)
        batched_np_data = [np.array_split(datum, len(np_data[0]) // BATCH_SIZE) for datum in np_data]
        batched_mem = list(zip(*batched_np_data))

        # Optimize
        models.optimize_actor_critic(actor_critic, optimize_mode, batched_mem, actor_critic_opt, tb_metrics)

        # Save weights for evaluation
        actor_critic.save_weights(TMP_WEIGHTS_PATH)
    
    # Evaluate against checkpoint model and other baselines
    rand_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, random_policy_args, 
                                       None, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    greed_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, greedy_policy_args, 
                                        None, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    opp_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, checkpoint_policy_args, 
                                      None, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

    stats = "{:.1f}%O, {:.1f}%R, {:.1f}%G".format(100*opp_win_rate, 
                                                 100*rand_win_rate,
                                                 100*greed_win_rate)

    # If it's better than the checkpoint, update
    if opp_win_rate > 0.6:
        actor_critic.save_weights(CHECKPOINT_PATH)
        print(f"{stats} Accepted new model")
    elif opp_win_rate > 0.5:
        print(f"{stats} Continuing to train current weights")
    else:
        actor_critic.load_weights(CHECKPOINT_PATH)
        print(f"{stats} Rejected new model")

    # Sync temp with checkpoint
    actor_critic.save_weights(TMP_WEIGHTS_PATH)

    # Log results and resets the metrics
    metrics.log_to_tensorboard(summary_writer, tb_metrics, iteration, go_env, 
                               actor_critic, 'logs/a_trajectory.jpg')

Updating:   0%|          | 0/49 [00:00<?, ?it/s]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Episodes:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Evaluate

Play against our AI

In [ ]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)
data.pit(go_env, black_policy=opponent_policy, white_policy=human_policy)